# VencoPy Tutorial 4

This tutorial aims to give a more in depth overview into the GridModeler class and showcases some features that can be customised.

In [2]:
import os, sys
import pandas as pd
import numpy as np
import yaml
import pathlib

path = '../..'
os.chdir(path)

#sys.path.append("../../classes")
#sys.path.append("../../../vencopy")

from classes.dataParsers import DataParser
from classes.tripDiaryBuilders import TripDiaryBuilder
from classes.gridModelers import GridModeler
from classes.flexEstimators import FlexEstimator
from classes.evaluators import Evaluator

print("Current working directory: {0}".format(os.getcwd()))

Current working directory: C:\8_Work\VencoPy\VencoPy_internal\vencopy


In [3]:
pathGlobalConfig = pathlib.Path.cwd() / 'config' / 'globalConfig.yaml'
with open(pathGlobalConfig) as ipf:
    globalConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathLocalPathConfig = pathlib.Path.cwd()  / 'config' / 'localPathConfig.yaml'
with open(pathLocalPathConfig) as ipf:
    localPathConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathParseConfig = pathlib.Path.cwd()  / 'config' / 'parseConfig.yaml'
with open(pathParseConfig) as ipf:
    parseConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathTripConfig = pathlib.Path.cwd()  / 'config' / 'tripConfig.yaml'
with open(pathTripConfig) as ipf:
    tripConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathGridConfig = pathlib.Path.cwd()  / 'config' / 'gridConfig.yaml'
with open(pathGridConfig) as ipf:
    gridConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathEvaluatorConfig = pathlib.Path.cwd()  / 'config' / 'evaluatorConfig.yaml'
with open(pathEvaluatorConfig) as ipf:
    evaluatorConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathFlexConfig = pathlib.Path.cwd()  / 'config' / 'flexConfig.yaml'
with open(pathFlexConfig) as ipf:
    flexConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
    
    
# Set reference dataset 
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
localPathConfig['pathAbsolute'][datasetID] = pathlib.Path.cwd() / 'tutorials' / 'data_sampling'

# Assign to vencoPyRoot the folder in which you cloned your repository
localPathConfig['pathAbsolute']['vencoPyRoot'] = pathlib.Path.cwd()

# Similarly we modify the datasetID in the global config file
globalConfig['files'][datasetID]['tripsDataRaw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del parseConfig['dataVariables']['hhID'] 
del parseConfig['dataVariables']['personID'] 

## GridModeler config file

The GridModeler config file 

In [4]:
gridConfig

{'chargingInfrastructureMappings': {'DRIVING': False,
  'HOME': True,
  'WORK': False,
  'SCHOOL': False,
  'SHOPPING': False,
  'LEISURE': False,
  'OTHER': False,
  'NA': False},
 'chargingInfrastructureDistributions': {'DRIVING': {1: 0},
  'HOME': {0.7: 3.6},
  'WORK': {0.5: 11},
  'SCHOOL': {0.5: 11},
  'SHOPPING': {0.5: 11},
  'LEISURE': {0.5: 11},
  'OTHER': {0.5: 11},
  'NA': {1: 0}}}

## _GridModeler_ class

In [5]:
# As before, the paths in the config file ned to be readjusted for the tutorials
globalConfig['pathRelative']['input'] = pathlib.Path.cwd() / globalConfig['pathRelative']['input']


In [6]:
vpGrid = GridModeler(gridConfig=gridConfig, globalConfig=globalConfig, datasetID=datasetID)
vpGrid.assignSimpleGridViaPurposes()
vpGrid.writeOutGridAvailability()

Starting with charge connection replacement of location purposes
Grid connection assignment complete


## Next Steps